In [1]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt


/home/mbergst/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
keypoints = {1: (44, 30), 2: (457, 30), 3: (44, 563), 4: (457, 563), 5: (44, 138), 6: (174, 138), 7: (174, 230), 8: (174, 346), 9: (174, 453), 10: (44, 453), 11: (44, 230), 12: (86, 230), 13: (86, 368), 14: (44, 368), 15: (130, 295), 16: (457, 230), 17: (457, 295), 18: (457, 368), 19: (201, 295), 20: (387, 295), 21: (530, 295), 22: (870, 30), 23: (870, 563), 24: (870, 138), 25: (740, 138), 26: (740, 230), 27: (740, 346), 28: (740, 453), 29: (870, 453), 30: (870, 230), 31: (827, 230), 32: (827, 368), 33: (870, 368), 34: (785, 295), 35: (712, 295)}

In [9]:
# Last inn sist trente modell
# Gjør det ikke bedre virker det som

def find_latest_model_with_best(base_path, type='keypoints'):
    detect_path = os.path.join(base_path, 'runs', type)
    training_sessions = [os.path.join(detect_path, d) for d in os.listdir(detect_path) if os.path.isdir(os.path.join(detect_path, d))]
    
    # Liste for å holde stiene til alle 'best.pt'-filer
    best_model_paths = []

    for session in training_sessions:
        best_model_path = os.path.join(session, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            best_model_paths.append((best_model_path, os.path.getmtime(best_model_path)))

    if not best_model_paths:
        print("Ingen 'best.pt' fil funnet i noen av treningsøktene.")
        return None
    
    # Sorter basert på modifiseringstid og returner den nyeste
    best_model_paths.sort(key=lambda x: x[1], reverse=True)
    return best_model_paths[0][0]


base_path = '/work/mbergst/TDT4265_Project' 


model = YOLO(find_latest_model_with_best(base_path, 'keypoints'))
#model = YOLO('yolov8n.pt')

In [3]:
results = model.train(data='yaml_files/keypoint.yaml', epochs=50, batch=20, imgsz=(1600, 900), project='/work/mbergst/TDT4265_Project/runs/keypoints')

New https://pypi.org/project/ultralytics/8.1.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
engine/trainer: task=detect, mode=train, model=/work/mbergst/TDT4265_Project/runs/keypoints/train34/weights/best.pt, data=yaml_files/keypoint.yaml, epochs=50, time=None, patience=100, batch=20, imgsz=(1600, 900), save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/mbergst/TDT4265_Project/runs/keypoints, name=train36, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fal

train: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/labels/train.cache... 1441 images, 0 backgrounds, 408 corrupt: 100%|██████████| 1441/1441 [00:00<?, ?it/s]

train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/train/000303.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/train/000304.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/train/000305.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
train: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/train/000306.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
train: WARNING ⚠️ /work/mbergst/TDT4265_


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/labels/val.cache... 361 images, 0 backgrounds, 91 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]

val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000314.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000318.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000319.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000324.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov

...

      50/50      17.5G     0.4633     0.3077     0.8128         67       1600: 100%|██████████| 52/52 [00:12<00:00,  4.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:00<00:00,  8.14it/s]

                   all        270       1643      0.877      0.885      0.903      0.759



50 epochs completed in 0.201 hours.
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/keypoints/train36/weights/last.pt, 6.4MB
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/keypoints/train36/weights/best.pt, 6.4MB

Validating /work/mbergst/TDT4265_Project/runs/keypoints/train36/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3012473 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.64it/s]


                   all        270       1643      0.879      0.884      0.903      0.759
                   kp2        270        270       0.99      0.974      0.994      0.833
                   kp6        270        132      0.969      0.985      0.993      0.832
                   kp7        270         73          1      0.938      0.994      0.775
                   kp8        270          1          0          0          0          0
                  kp16        270        260      0.988      0.985      0.995      0.909
                  kp18        270        259      0.994          1      0.995      0.929
                  kp19        270         71       0.97      0.898      0.981      0.769
                  kp20        270        270      0.996          1      0.995      0.918
                  kp21        270        270          1      0.986      0.995       0.91
                  kp26        270         36      0.945      0.953      0.991      0.876
                  kp2

In [5]:
# Evaluér modellens prestasjon på valideringssettet
val_results = model.val()

Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3012473 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/labels/val.cache... 361 images, 0 backgrounds, 91 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]

val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000314.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000318.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000319.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000324.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00, 10.39it/s]


                   all        270       1643      0.761      0.756      0.784      0.573
                   kp2        270        270      0.962      0.963      0.979      0.642
                   kp6        270        132      0.899      0.875      0.946      0.661
                   kp7        270         73      0.898      0.918       0.94      0.517
                   kp8        270          1          0          0          0          0
                  kp16        270        260      0.977      0.962      0.993      0.828
                  kp18        270        259       0.97      0.996      0.994      0.853
                  kp19        270         71       0.83      0.789      0.836      0.476
                  kp20        270        270      0.987      0.956      0.992      0.831
                  kp21        270        270      0.983      0.967      0.992      0.824
                  kp26        270         36      0.863      0.889      0.947      0.669
                  kp2

In [10]:
# Tracking

all_frames = '/work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames'
frame_paths = sorted([os.path.join(all_frames, f) for f in os.listdir(all_frames) if f.endswith('.jpg')])


In [12]:
# Iterere over hver frame og bruk track metoden
for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_width=1):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene


image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000001.jpg: 928x1600 1 kp16, 1 kp20, 2.9ms
Speed: 3.6ms preprocess, 2.9ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000002.jpg: 928x1600 1 kp2, 1 kp16, 1 kp18, 1 kp20, 1 kp21, 3.0ms
Speed: 3.3ms preprocess, 3.0ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000003.jpg: 928x1600 1 kp2, 1 kp16, 1 kp18, 1 kp20, 1 kp21, 2.9ms
Speed: 3.8ms preprocess, 2.9ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000004.jpg: 928x1600 1 kp2, 1 kp16, 1 kp18, 1 kp20, 1 kp21, 3.0ms
Speed: 3.4ms pre